In [62]:
import os
import json
import numpy as np
import cv2
from PIL import Image
from pycocotools import mask as maskUtils

# Analyze no. person videos

## 0. Overlapping between YTVIS2019 and TYTVIS2021?

In [7]:
ytvis2019_image_dir = "/home/chuongh/vm2m/data/YTVIS/2019/train/JPEGImages"
ytvis2021_image_dir = "/home/chuongh/vm2m/data/YTVIS/2021/train/JPEGImages"
diff_list = set(list(os.listdir(ytvis2019_image_dir))).difference(set(list(os.listdir(ytvis2021_image_dir))))

In [5]:
len(list(os.listdir(ytvis2019_image_dir)))

3471

In [6]:
len(list(os.listdir(ytvis2021_image_dir)))

2985

In [9]:
len(diff_list)

1280

## 1. How many videos with people?

In [32]:
person_videos = set()

# Process 2019 videos
data = json.load(open("/home/chuongh/vm2m/data/YTVIS/2019/train.json", "r"))
videoId2Name = {}
for video in data["videos"]:
    videoId2Name[video["id"]] = video["file_names"][0].split("/")[0]

for anno in data['annotations']:
    cate = anno['category_id']
    if cate == 1:
        person_videos.add(videoId2Name[anno['video_id']])

In [35]:
person_videos_2021 = set()
data = json.load(open("/home/chuongh/vm2m/data/YTVIS/2021/train/instances.json", "r"))
videoId2Name = {}
for video in data["videos"]:
    videoId2Name[video["id"]] = video["file_names"][0].split("/")[0]
for anno in data['annotations']:
    cate = anno['category_id']
    if cate == 26:
        person_videos_2021.add(videoId2Name[anno['video_id']])

In [36]:
len(person_videos_2021)

1180

In [37]:
for x in person_videos_2021:
    person_videos.add(x)

In [38]:
len(person_videos)

1180

Person videos in YTVIS: 1180

In [39]:
with open("/home/chuongh/vm2m/data/YTVIS/person_videos.txt", "w") as f:
    for x in list(person_videos):
        f.write(str(x) + "\n")

In [45]:
# Process OVIS videos
data = json.load(open("/home/chuongh/vm2m/data/OVIS/annotations_train.json", "r"))
videoId2Name = {}
person_videos_ovis = set()
for video in data["videos"]:
    videoId2Name[video["id"]] = video["file_names"][0].split("/")[0]
for anno in data['annotations']:
    cate = anno['category_id']
    if cate == 1:
        person_videos_ovis.add(videoId2Name[anno['video_id']])

In [46]:
len(person_videos_ovis)

181

Person videos in OVIS: 181

In [47]:
with open("/home/chuongh/vm2m/data/OVIS/person_videos.txt", "w") as f:
    for x in list(person_videos_ovis):
        f.write(str(x) + "\n")

## 3. How many frames/video?

In [48]:
n_frames = 0
for video_name in list(person_videos):
    video_dir = os.path.join(ytvis2021_image_dir, video_name)
    if not os.path.exists(video_dir):
        video_dir = os.path.join(ytvis2019_image_dir, video_name)
    n_frames += len(os.listdir(video_dir))

In [49]:
n_frames/ len(person_videos)

30.608474576271185

YTVIS frame rate: 30frame/video

In [50]:
n_frames = 0
for video_name in list(person_videos_ovis):
    video_dir = os.path.join("/home/chuongh/vm2m/data/OVIS/train", video_name)
    n_frames += len(os.listdir(video_dir))

In [51]:
n_frames/ len(person_videos_ovis)

74.48066298342542

OVIS frame rate: 75 frame/video

In [71]:
# Save annotation of person videos

# For YTVIS 2021
out_dir = "/home/chuongh/vm2m/data/YTVIS/2021/train/masks"
data = json.load(open("/home/chuongh/vm2m/data/YTVIS/2021/train/instances.json", "r"))
videoId2Name = {}
videoId2FileNames = {}
for video in data["videos"]:
    videoId2Name[video["id"]] = video["file_names"][0].split("/")[0]
    videoId2FileNames[video["id"]] = video["file_names"]
videoId2NoInst = {}
for anno in data["annotations"]:
    video_name = videoId2Name[anno["video_id"]]
    cate = anno["category_id"]
    if cate != 26 or video_name not in person_videos:
        continue
    width, height = anno["width"], anno["height"]
    segms = anno["segmentations"]
    frame_names = videoId2FileNames[anno["video_id"]]
    videoId2NoInst[anno["video_id"]] = videoId2NoInst.get(anno["video_id"], 0) + 1
    inst_id = videoId2NoInst[anno["video_id"]] - 1
    for seg, frame_name in zip(segms, frame_names):
        if seg is None:
            mask = np.zeros((height, width), dtype=np.uint8)
        else:
            mask = maskUtils.decode(maskUtils.frPyObjects(seg, height, width))
        
        out_name = frame_name.replace(".jpg", "")
        out_path = os.path.join(out_dir, out_name, f"{inst_id:02d}.png")
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        mask = Image.fromarray(mask * 255)
        mask.save(out_path)

In [69]:
frame_names[0]

'01c4cb5ffe/00000.jpg'